In [1]:
import os
import csv
import gzip
import glob
import pathlib
import argparse
import datetime

In [2]:
import logging
########## logging
# create logger with 'spam_application'
logger = logging.getLogger('notebook')
logger.setLevel(logging.DEBUG)

# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s]: %(message)s')
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(ch)
##########

In [3]:
import progressbar
progressbar.streams.wrap_stderr()

[2018-05-22 23:04:51,415][DEBUG]: input_files_count: 24
[2018-05-22 23:04:51,471][DEBUG]: input_file: ../data/test/pagecounts-20071210-070000.gz
[2018-05-22 23:04:51,474][INFO]: Processing file: ../data/test/pagecounts-20071210-070000.gz
[2018-05-22 23:04:53,553][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-070000.gz to list
[2018-05-22 23:04:53,553][DEBUG]: input_file: ../data/test/pagecounts-20071210-200000.gz
[2018-05-22 23:04:53,555][INFO]: Processing file: ../data/test/pagecounts-20071210-200000.gz
[2018-05-22 23:04:53,652][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-200000.gz to list
[2018-05-22 23:04:53,653][DEBUG]: input_file: ../data/test/pagecounts-20071210-000000.gz
[2018-05-22 23:04:53,654][INFO]: Processing file: ../data/test/pagecounts-20071210-000000.gz
[2018-05-22 23:04:53,704][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-000000.gz to list
[2018-05-22 23:04:53,706][DEBUG]: input_file: ../data/test/pagecounts-2

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, TimestampType
from pyspark.sql import functions
from pyspark.sql.functions import lit

In [6]:
sc = pyspark.SparkContext(appName="merge-pagecounts")
sqlctx = pyspark.SQLContext(sc)

In [7]:
schema = StructType([StructField("lang", StringType(), False),
                     StructField("page", StringType(), False),
                     StructField("views", IntegerType(), False),
                     StructField("reqbytes", IntegerType(), False)])

In [8]:
def unionAll(*dfs):
    first, *_ = dfs  # Python 3.x, for 2.x you'll have to unpack manually
    return first.sql_ctx.createDataFrame(
        first.sql_ctx._sc.union([df.rdd for df in dfs]),
        first.schema
    )


def date_parser(timestamp):
    return datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M%S')

In [9]:
pathfile = "../data/test/pagecounts-20071210-*.gz"

In [10]:
input_files_count = len([f for f in glob.iglob(pathfile)])
input_files = glob.iglob(pathfile)

# input_files = ["data/input/sorted_time/2007-12/pagecounts-20071210-000000.gz",
#                "data/input/sorted_time/2007-12/pagecounts-20071210-010000.gz"
#                ]
# input_files_count = len(input_files)

logger.debug('input_files_count: {}'.format(input_files_count))

In [11]:
if input_files_count < 1:
    logger.warn('No input files match: exiting')
    exit(0)

In [12]:
input_files_count

24

In [13]:
list_dfs = list()
with progressbar.ProgressBar(max_value=input_files_count) as bar:
    for input_file in input_files:
        logger.debug('input_file: {}'.format(input_file))

        timestamp = date_parser(os.path.basename(input_file)
                                       .replace('pagecounts-','')
                                       .replace('.gz',''))

        logger.info('Processing file: {}'.format(input_file))
        tmp_spark_df = sqlctx.read.csv(
                            input_file,
                            header=False,
                            schema=schema,
                            sep=' ')

        tmp_spark_df = tmp_spark_df.withColumn("timestamp", lit(timestamp))
        list_dfs.append(tmp_spark_df)
        del tmp_spark_df

        logger.info('Added DataFrame for file {} to list'.format(input_file))

In [14]:
assert len(list_dfs) >= 1, 'There should be at least one DataFrame'

In [15]:
if len(list_dfs) > 1:
    logger.info('Union of all Spark DataFrames.')
    df = unionAll(*list_dfs)
    logger.info('Spark DataFrame created')
else:
    df = list_dfs[0]

In [16]:
df.count()

286

In [17]:
logger.info('Dropping column "reqbytes" from DataFrame')
df = df.drop('reqbytes')
logger.info('Dropped column "reqbytes" from DataFrame')

In [18]:
df.dtypes

[('lang', 'string'),
 ('page', 'string'),
 ('views', 'int'),
 ('timestamp', 'timestamp')]

In [19]:
grouped_daily_df = (df.select(['lang',
                               'page',
                               functions.date_format('timestamp','yyyy-MM-dd')\
                                        .alias('day'),
                               'views'])
                      .groupby(['lang','page','day'])
                      .sum('views')
                      )

In [20]:
grouped_daily_df.show(10)

+----+--------------------+----------+----------+
|lang|                page|       day|sum(views)|
+----+--------------------+----------+----------+
|  en|Albert_Cardinal_D...|2007-12-10|         1|
|  en| Albert_Caesar_Tocco|2007-12-10|         2|
|  en|    Albert_C__Outler|2007-12-10|         1|
|  en|Albert_Camus#Furt...|2007-12-10|         1|
|  en|Albert_Camuscolum...|2007-12-10|         1|
|  en|    Albert_Carnesale|2007-12-10|        12|
|  en|        Albert_Buick|2007-12-10|         1|
|  en|        Albert_Camus|2007-12-10|      2908|
|  en|      Albert_Bunjaku|2007-12-10|         1|
|  en|        Albert_Burgh|2007-12-10|         1|
+----+--------------------+----------+----------+
only showing top 10 rows



In [21]:
new_schema = StructType([StructField("lang", StringType(), False),
                         StructField("page", StringType(), False),
                         StructField("day", StringType(), False),
                         StructField("enc", StringType(), False)])

In [22]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd

hour_to_letter = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
                  'P','Q','R','S','T','U','V','W','X']

@pandas_udf(new_schema, PandasUDFType.GROUPED_MAP)
def concat_hours(x):
    view_hours = x['hour'].tolist()
    view_views = x['views'].tolist()

    view_hours_letters = [hour_to_letter[h] for h in view_hours]

    encoded_views = [l + str(h)
                     for l, h in sorted(zip(view_hours_letters,view_views))]
    encoded_views_string = ''.join(encoded_views)

    # return pd.DataFrame({'page': x.page, 'lang': x.lang,'day': x.day, 'enc': encoded_views_string}, index=[x.index[0]])
    return pd.DataFrame({'enc': x.page, 'day': x.lang, 'lang': x.day, 'page': encoded_views_string}, index=[x.index[0]])

In [23]:
from pyspark.sql import functions
grouped_hours_df = (df.select(['lang',
                               'page',
                               functions.date_format('timestamp','yyyy-MM-dd').alias('day'), 
                               functions.hour('timestamp').alias('hour'), 
                               'views'
                               ])
                      .groupby(['lang','page','day'])
                      )

In [24]:
grouped_hours_df = (grouped_hours_df.apply(concat_hours)
                                    .dropDuplicates()
                                    )

In [25]:
grouped_hours_df.show()

+----+--------------------+----------+--------------------+
|lang|                page|       day|                 enc|
+----+--------------------+----------+--------------------+
|  en|   Albert_C._Ritchie|2007-12-10|            Q2S2V1W1|
|  en|Albert_C._L._G._G...|2007-12-10|                  J1|
|  en|      Albert_Calland|2007-12-10|        B1D3I1L1O1Q1|
|  en|       Albert_Brooks|2007-12-10|           G16K11N13|
|  en|    Albert_C1984amus|2007-12-10|                  V1|
|  en| Albert_C._Wedemeyer|2007-12-10|                B1X1|
|  en|       Albert_Chowne|2007-12-10|                  R1|
|  en|        Albert_Camus|2007-12-10|A150B148C197D173E...|
|  en|Albert_Cardinal_V...|2007-12-10|                  E1|
|  en|    Albert_C__Barnes|2007-12-10|                  U1|
|  en|Albert_Camus#Oppo...|2007-12-10|                  E1|
|  en|     Albert_Calmette|2007-12-10|B1C1F1H1K1L1O1P2R...|
|  en|Albert_Burnley_Botts|2007-12-10|                  M1|
|  en|Albert_Camus_-_Wi...|2007-12-10|  

In [26]:
grouped_daily_df.show(10)

+----+--------------------+----------+----------+
|lang|                page|       day|sum(views)|
+----+--------------------+----------+----------+
|  en|Albert_Cardinal_D...|2007-12-10|         1|
|  en| Albert_Caesar_Tocco|2007-12-10|         2|
|  en|    Albert_C__Outler|2007-12-10|         1|
|  en|Albert_Camus#Furt...|2007-12-10|         1|
|  en|Albert_Camuscolum...|2007-12-10|         1|
|  en|    Albert_Carnesale|2007-12-10|        12|
|  en|        Albert_Buick|2007-12-10|         1|
|  en|        Albert_Camus|2007-12-10|      2908|
|  en|      Albert_Bunjaku|2007-12-10|         1|
|  en|        Albert_Burgh|2007-12-10|         1|
+----+--------------------+----------+----------+
only showing top 10 rows



In [27]:
from pyspark.sql.functions import col
daily = grouped_daily_df.select([col('lang').alias('daily_lang'),
                                 col('page').alias('daily_page'),
                                 col('day').alias('daily_day'),
                                 col('sum(views)').alias('daily_sum_views'),                                 
                                 ])

In [28]:
daily.show()

+----------+--------------------+----------+---------------+
|daily_lang|          daily_page| daily_day|daily_sum_views|
+----------+--------------------+----------+---------------+
|        en|Albert_Cardinal_D...|2007-12-10|              1|
|        en| Albert_Caesar_Tocco|2007-12-10|              2|
|        en|    Albert_C__Outler|2007-12-10|              1|
|        en|Albert_Camus#Furt...|2007-12-10|              1|
|        en|Albert_Camuscolum...|2007-12-10|              1|
|        en|    Albert_Carnesale|2007-12-10|             12|
|        en|        Albert_Buick|2007-12-10|              1|
|        en|        Albert_Camus|2007-12-10|           2908|
|        en|      Albert_Bunjaku|2007-12-10|              1|
|        en|        Albert_Burgh|2007-12-10|              1|
|        en|      Albert_Cadwell|2007-12-10|              1|
|        en|       Albert_Caquot|2007-12-10|              8|
|        en|     Albert_C._Field|2007-12-10|              2|
|        en|Albert_Camus

In [29]:
cond = [daily.daily_lang == grouped_hours_df.lang,
        daily.daily_page == grouped_hours_df.page,
        daily.daily_day == grouped_hours_df.day]
final = daily.join(grouped_hours_df, cond).select(['daily_lang','daily_page','daily_day', 'daily_sum_views', 'enc'])

In [30]:
final.dropDuplicates().show()

+----------+--------------------+----------+---------------+--------------------+
|daily_lang|          daily_page| daily_day|daily_sum_views|                 enc|
+----------+--------------------+----------+---------------+--------------------+
|        en|Albert_Cardinal_D...|2007-12-10|              1|                  Q1|
|        en|    Albert_C__Outler|2007-12-10|              1|                  R1|
|        en| Albert_Caesar_Tocco|2007-12-10|              2|                N1P1|
|        en|Albert_Camus#Furt...|2007-12-10|              1|                  E1|
|        en|Albert_Camuscolum...|2007-12-10|              1|                  J1|
|        en|    Albert_Carnesale|2007-12-10|             12|          G4H5L1V1X1|
|        en|        Albert_Buick|2007-12-10|              1|                  K1|
|        en|      Albert_Bunjaku|2007-12-10|              1|                  A1|
|        en|        Albert_Burgh|2007-12-10|              1|                  M1|
|        en|    